# Papirbro

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sms
import statsmodels.formula.api as smf
from scipy import stats

In [ ]:
run DOEfunksjoner.ipynb 

## DEL 1

In [ ]:
X = tofaktorDOE(k=3)
print(X)

In [ ]:
y1 = [8.7, 17.4, 0, 8.7, 8.7, 30.45, 4.35, 13.05]
y2 = [4.35, 17.4, 4.35, 8.7, 8.7, 34.8, 4.35, 21.75]

In [ ]:
tab = np.column_stack([X, y1,y2])
colnames = ["x1", "x2","x3", "y1","y2"]
df = pd.DataFrame(tab,columns=colnames)
print(df)

In [ ]:
colnames = ["x1", "x2","x3", "y"]
rep1 = pd.DataFrame(np.column_stack([X, y1]),columns=colnames)
rep2 = pd.DataFrame(np.column_stack([X, y2]),columns=colnames)

df = pd.concat([rep1,rep2])

print(df)

In [ ]:
modell = smf.ols('y~x1+x2+x3 + x1*x2 + x2*x3 + x1*x3', data=df).fit()
print(modell.summary())

In [ ]:
DOEparetoplott(modell)

In [ ]:
hovedeffektsplott(data = df, k = 3, observasjoner = True)

In [ ]:
samspillsplott(data = df, k = 3)

In [ ]:
sms.qqplot(modell.resid,line='45',fit=True)
plt.ylabel("Kvantiler i residualene")
plt.xlabel("Kvantiler i normalfordelingen")
plt.show()

## DEL 2

In [ ]:
varY = modell.mse_resid
sdY = np.sqrt(varY)
print(round(sdY,2))

In [ ]:
modell.mse_resid

In [ ]:
modell.predict()

## Stokastisk simulering
Her simulerer jeg 100 broer fra en produksjon som er under kontroll (dere skal ikke gjøre dette i prosjektet, dette er bare for å visulisere hvordan en slik prosess ser ut)

In [ ]:
mean = 31.81
std_dev = 2.92
antall_broer = 100
bæreevne = np.zeros(antall_broer)

for i in range(antall_broer):
    bæreevne[i] = np.random.normal(mean, std_dev)

bro_nr = np.arange(1, antall_broer + 1)

plt.scatter(bro_nr, bæreevne, marker='o',color='b')
plt.ylim(10,60); plt.xlabel("Bro nr."); plt.ylabel("Observert bæreevne")
plt.grid()
plt.show()

Her simulerer jeg 100 broer fra en produksjon som ikke er under kontroll (dere skal ikke gjøre dette i prosjektet, dette er bare for å visulisere hvordan en slik prosess ser ut). Her endrer forventningsverdien seg med $- 0.2\cdot \sigma$ for hver femte bro som produseres

In [ ]:
mean = 31.81
std_dev = 2.92
antall_broer = 100
bæreevne = np.zeros(antall_broer)

for i in range(antall_broer):
    if (i + 1) % 5 == 0:
        mean -= 0.3 * std_dev
    bæreevne[i] = np.random.normal(mean, std_dev)

bro_nr = np.arange(1, antall_broer + 1)

plt.scatter(bro_nr, bæreevne, marker='o',  color='b')
plt.ylim(0,60); plt.xlabel("Bro nr."); plt.ylabel("Observert bæreevne")
plt.grid()
plt.show()

Her simulerer jeg 100 broer fra en produksjon som ikke er under kontroll (dere skal ikke gjøre dette i prosjektet, dette er bare for å visulisere hvordan en slik prosess ser ut). Her endrer standardavviket seg med $+ 0.5$ for hver femte bro som produseres

In [ ]:
mean = 31.81
std_dev = 2.92
antall_broer = 100
bæreevne = np.zeros(antall_broer)

for i in range(antall_broer):
    if (i + 1) % 5 == 0:
        std_dev += 0.5
    bæreevne[i] = np.random.normal(mean, std_dev)

bro_nr = np.arange(1, antall_broer + 1)

plt.scatter(bro_nr, bæreevne, marker='o', color='b')
plt.ylim(0,60); plt.xlabel("Bro nr."); plt.ylabel("Observert bæreevne")
plt.grid()
plt.show()

### Kontrollgrenser og stikkprøver
MERK: Dere skal gjøre noe som ligner på dette i del 2 av prosjektet, men dere må gjøre noen endringer i koden!

In [ ]:
mean = 31.81
std_dev = 2.92

n_stikkprøve = 5
cu_mean = mean + 3*std_dev/np.sqrt(n_stikkprøve)
cl_mean = mean - 3*std_dev/np.sqrt(n_stikkprøve)

print(cu_mean)
print(cl_mean)

cu_std = std_dev*np.sqrt(stats.chi2.ppf(1-0.002,(n_stikkprøve-1))/(n_stikkprøve-1))
cl_std = std_dev*np.sqrt(stats.chi2.ppf(0.002,(n_stikkprøve-1))/(n_stikkprøve-1))
print(cu_std)
print(cl_std)


In [ ]:
stikkprøver = 50
mean = 31.81
std_dev = 2.92

snitt_bæreevne = np.zeros(stikkprøver)
std_bæreevne = np.zeros(stikkprøver)

for i in range(stikkprøver):
    utvalg = np.random.normal(mean, std_dev, n_stikkprøve)
    snitt_bæreevne[i] = np.mean(utvalg)
    std_bæreevne[i] = np.std(utvalg, ddof=1)

stikkprøve_nr = np.arange(1, stikkprøver + 1)

plt.scatter(stikkprøve_nr, snitt_bæreevne, marker='o', color='b')
plt.ylim(10,50); plt.xlabel("Stikkprøve nr."); plt.ylabel("Gjennomsnittlig bæreevne")
plt.axhline(y=cu_mean, color='r', linestyle='--'); plt.axhline(y=cl_mean, color='r', linestyle='--')
plt.grid()
plt.show()

Over ser vi en simuleringer av stikkprøver basert på 5 broer av gangen. Vi har plottet oververt gjennomsnittlig bæreevne for de fem broene i hver stikkprøve. Prosessen er under kontroll. Kontrollgrensene er tegnet som røde linjer. Dette er et Shewarts $\bar{x}$-diagram.

Sammenligne gjerne med figuren som viste simulering av 100 broer. Ser dere at variasjonen er mye lavere nå som vi ser på fem broer av gangen, i stedet for en og en?

Under ser du et $s$-diagram for standardavviket i kontrollprøvene. Her ser vi hvordan empirisk standardavvik i hver stikkprøve fordeler seg når prosessen er under kontroll. Kontrollgrensene for standardavviket er tegnet som røde linjer.

In [ ]:
plt.scatter(stikkprøve_nr, std_bæreevne, marker='o', color='b')
plt.ylim(0,8); plt.xlabel("Stikkprøve nr."); plt.ylabel("Empirisk standardavvik bæreevne")
plt.axhline(y=cu_std, color='r', linestyle='--')
plt.axhline(y=cl_std, color='r', linestyle='--')
plt.grid()
plt.show()

### Stokastisk simulering av stikkprøver når parametere endrer seg
I simuleringen under har jeg kodet inn at forventningsverdien endrer seg med $+0.5$ for hver femte stikkprøve. Jeg har også kodet inn at standardavviket endrer seg med $+0.1$ for hver femte stikkprøve. Vi observerer hva som skjer i $\bar{x}$- og $s$-diagrammer. 

*MERK!* Dere skal ikke gjøre akkurat det samme i prosjektet, tilpass simuleringer til oppgavebeskrivelsen.

Kjør gjerne simuleringen flere ganger for å se hva som skjer. Når går alarmen? Ser vi utslagene først i $\bar{x}$-diagrammet eller ${s}$-diagrammet? (dette kan endre seg mellom simuleringer)

In [ ]:
stikkprøver = 50

mean = 31.81
std_dev = 2.92

snitt_bæreevne = np.zeros(stikkprøver)
std_bæreevne = np.zeros(stikkprøver)

for i in range(stikkprøver):
    if (i + 1) % 5 == 0:
        mean += 0.5
        std_dev += 0.1
    utvalg = np.random.normal(mean, std_dev, n_stikkprøve)
    snitt_bæreevne[i] = np.mean(utvalg)
    std_bæreevne[i] = np.std(utvalg, ddof=1)


stikkprøve_nr = np.arange(1, stikkprøver + 1)

plt.scatter(stikkprøve_nr, snitt_bæreevne, marker='o', color='b')
plt.ylim(10,50); plt.xlabel("Stikkprøve nr."); plt.ylabel("Gjennomsnittlig bæreevne")
plt.axhline(y=cu_mean, color='r', linestyle='--')
plt.axhline(y=cl_mean, color='r', linestyle='--')
plt.grid()
plt.show()

plt.scatter(stikkprøve_nr, std_bæreevne, marker='o', color='b')
plt.ylim(0,8); plt.xlabel("Stikkprøve nr."); plt.ylabel("Empirisk standardavvik bæreevne")
plt.axhline(y=cu_std, color='r', linestyle='--')
plt.axhline(y=cl_std, color='r', linestyle='--')
plt.grid()
plt.show()